# Getting locations with Google Maps API
So part 2 of our project, we need to get information about the location, or coordinates of the respective football clubs in the EFL Championship, in order for us to plot them accurately on a UK map.

To do this, I've used the Google Maps Directions API.

In [1]:
import requests

As before, I've chosen to use the `requests` module to send a GET() request to the API endpoint of the Google Maps API.

### What is an API?
Before we continue, an API, which stands for Application Programming Interface, is simply a way you can speak to other applications. 

For a quick 3 minute video overview on what an API is, I'd recommend [this one](https://youtu.be/s7wmiS2mSXY).

So for Google Maps, we are essentially sending a request to find out information about how to get between an origin, and a destination, and as long as we follow the syntax that their API requires, and provide it an access or API key (so they know it is me and not someone abusing their systems), they will behind the scenes process the request and return the useful information! Voila!

### Setting up a Google Developer Account

* Navigate to [The Google Cloud - Maps Platform](https://cloud.google.com/maps-platform/) and sign up
* Unfortunately you do have to give a debit/credit card, but you get $300 of credit to use, and unless you're building a full-blown application, it is very unlikely you'll hit the limit enough to cost you money!
* Once you've done that, you'll want to enable a specific API credential, for the Google Maps Directions API
* You'll then receive an API Key, which is your SECRET key to authenticate with Google Maps, so they know it is you.


### Building a function to query the API
Once you have the API Key, you can go about building a function to query this API and capture the response.

To understand how to structure your query/request, generally you can just read the documentation. Let's take a look [here](https://developers.google.com/maps/documentation/directions/overview).

In [36]:
def get_directions(origin: str, destination: str):
    """
    This function will take in an origin and a destination, query the Google
    Maps API and then return the json response in dictionary form.

    :param origin: The start location from where you will be travelling from.
    :param destination: The location you want to travel to.
    :return: Dictionary of direction information
    """
    API_KEY = ""
    endpoint = "https://maps.googleapis.com/maps/api/directions/json?"
    nav_request = f"origin={origin}&destination={destination}&key={API_KEY}"
    request = endpoint + nav_request
    response = requests.get(request).json()
    
    return response

In [37]:
directions = get_directions("Nottingham", "Bournemouth")

In [44]:
directions['routes'][0]['legs'][0]['distance']['value']

313988

### Helper Functions
Okay now that we've got a really helpful function that pulls all the useful information about directions between 2 locations, we will now create a handful of [modular](https://en.wikipedia.org/wiki/Modular_programming#:~:text=Modular%20programming%20is%20a%20software,aspect%20of%20the%20desired%20functionality.), helper functions that each serve their own purpose.

1. Function that handles the response data to get the distance between 2 points
1. Function that handles the response data to get the latitude and longitude between 2 points
1. Function to convert metres to kilometres, and round-up.


In [20]:
def get_distance(directions):
    return directions['routes'][0]['legs'][0]['distance']['value']

def get_lat_long(directions):
    end_location = directions['routes'][0]['legs'][0]['end_location']
    latitude = end_location['lat']
    longitude = end_location['lng']
    return (latitude, longitude)

from math import ceil
def convert_distance_m_to_km(distance):
    distance_km = ceil(distance / 1000)
    return distance_km

In [45]:
distance = get_distance(directions)
distance

313988

In [46]:
lat_long = get_lat_long(directions)
lat_long

(50.71915509999999, -1.880781)

In [47]:
convert_distance_m_to_km(distance)

314

### Applying it all to our EFL team data

In [48]:
import efl_mapping as efl

In [49]:
df = efl.get_efl_team_data()

In [50]:
df.head()

,Club,Stadium
0,Barnsley,Oakwell
1,Birmingham City,St Andrew's
2,Blackburn Rovers,Ewood Park
3,Bournemouth,Dean Court
4,Brentford,Brentford Community Stadium


In [51]:
import pandas as pd

origin = "Nottingham"
stadium_list = {}

df = efl.get_efl_team_data()

for stadium, club in zip(df.Stadium, df.Club):
    print(stadium, club)
    destination = stadium + ' ' + club
    # destination = replace_space_with_plus(destination)
    directions = get_directions(origin, destination)
    distance = get_distance(directions)
    lat_long = get_lat_long(directions)
    stadium_list[stadium] = {
        "distance": convert_distance_m_to_km(distance),
        "latitude": lat_long[0],
        "longitude": lat_long[1],
    }
    
lat_long_df = pd.DataFrame.from_dict(stadium_list).T
df = df.merge(lat_long_df, how='inner', left_on='Stadium', right_index=True)

Oakwell Barnsley
St Andrew's Birmingham City
Ewood Park Blackburn Rovers
Dean Court Bournemouth
Brentford Community Stadium Brentford
Ashton Gate Bristol City
Cardiff City Stadium Cardiff City
St Andrew's Coventry City
Pride Park Stadium Derby County
Kirklees Stadium Huddersfield Town
Kenilworth Road Luton Town
Riverside Stadium Middlesbrough
The Den Millwall
Carrow Road Norwich City
City Ground Nottingham Forest
Deepdale Preston North End
Loftus Road Queens Park Rangers
Madejski Stadium Reading
New York Stadium Rotherham United
Hillsborough Sheffield Wednesday
bet365 Stadium Stoke City
Liberty Stadium Swansea City
Vicarage Road Watford
Adams Park Wycombe Wanderers


In [52]:
df

,Club,Stadium,distance,latitude,longitude
0,Barnsley,Oakwell,86.0,53.551975,-1.469151
1,Birmingham City,St Andrew's,84.0,52.476337,-1.867842
7,Coventry City,St Andrew's,84.0,52.476337,-1.867842
2,Blackburn Rovers,Ewood Park,198.0,53.728238,-2.490504
3,Bournemouth,Dean Court,310.0,50.734809,-1.840450
4,Brentford,Brentford Community Stadium,206.0,51.489809,-0.287871
5,Bristol City,Ashton Gate,229.0,51.441430,-2.620553
6,Cardiff City,Cardiff City Stadium,271.0,51.473613,-3.202064
8,Derby County,Pride Park Stadium,23.0,52.914314,-1.448229
9,Huddersfield Town,Kirklees Stadium,108.0,53.655047,-1.768938
